# Analyse scenarios

We focus on temperature deltas arising from notebook 200 compared to "baseline"

In [ ]:
import h5py
import matplotlib.pyplot as pl
import numpy as np
from climateforcing.utils import mkdir_p

In [ ]:
def load_dict_from_hdf5(filename):
    """
    ....
    """
    with h5py.File(filename, 'r') as h5file:
        return recursively_load_dict_contents_from_group(h5file, '/')

def recursively_load_dict_contents_from_group(h5file, path):
    """
    ....
    """
    ans = {}
    for key, item in h5file[path].items():
        if isinstance(item, h5py._hl.dataset.Dataset):
            ans[key] = item[:]
        elif isinstance(item, h5py._hl.group.Group):
            ans[key] = recursively_load_dict_contents_from_group(h5file, path + key + '/')
    return ans

In [ ]:
results_in = load_dict_from_hdf5('../data_output_large/fair-ensemble.h5')

In [ ]:
results_in['baseline']['HFC125 radiative forcing'][260:, 0]

In [ ]:
results_in['ssp245']['HFC125 radiative forcing'][260:, 0]

In [ ]:
scenarios = ["ssp245", "baseline", "hfc-32", "propane-as-slcf"]
variables = ['temperature (relative to 1850-1900)', 'HFC32 radiative forcing', 'HFC125 radiative forcing',
      'HCFC22 radiative forcing',
       'O3 radiative forcing',
       'total radiative forcing']

In [ ]:
fig, ax = pl.subplots(2, 3, figsize=(16,10))

colors = {
    "hfc-32": 'tab:orange',
    "propane-as-slcf": 'tab:blue', 
}

for ivar, variable in enumerate(variables):
    for scenario in scenarios[2:]:
        ax[ivar//3, ivar%3].fill_between(
            np.arange(2010, 2101), 
            np.percentile(results_in[scenario][variable][260:351,:]-results_in['baseline'][variable][260:351,:], 5, axis=1),
            np.percentile(results_in[scenario][variable][260:351,:]-results_in['baseline'][variable][260:351,:], 95, axis=1),
            color=colors[scenario],
            alpha=0.3
        )
        ax[ivar//3, ivar%3].plot(
            np.arange(2010, 2101), 
            np.median(results_in[scenario][variable][260:351,:]-results_in['baseline'][variable][260:351,:], axis=1),
            label=scenario,
            color=colors[scenario]
        )
        ax[ivar//3, ivar%3].set_title(variable)
ax[0,0].legend()

mkdir_p('../plots/')
fig.set_facecolor('white')
pl.savefig('../plots/scenario-comparisons.png')

In [ ]:
pl.rcParams['font.size'] = 11
pl.rcParams['figure.dpi'] = 300
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['ytick.direction'] = 'out'
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['figure.facecolor'] = 'white'

In [ ]:
fig, ax = pl.subplots(1, 1, figsize=(9/2.54,9/2.54))

scen_names = {
    "hfc-32": 'HFC-32 scenario',
    "propane-as-slcf": 'Propane scenario', 
}

variable = 'temperature (relative to 1850-1900)'
for scenario in scenarios[2:]:
    ax.fill_between(
        np.arange(2010, 2101), 
        np.percentile(results_in[scenario][variable][260:351,:]-results_in['baseline'][variable][260:351,:], 5, axis=1),
        np.percentile(results_in[scenario][variable][260:351,:]-results_in['baseline'][variable][260:351,:], 95, axis=1),
        color=colors[scenario],
        alpha=0.3
    )
    ax.plot(
        np.arange(2010, 2101), 
        np.median(results_in[scenario][variable][260:351,:]-results_in['baseline'][variable][260:351,:], axis=1),
        label=scen_names[scenario],
        color=colors[scenario]
    )
    ax.set_title('Global mean temperature difference')
ax.legend()

ax.axhline(0, ls=':', color='k', lw=0.5)
ax.set_ylim(-0.13, 0.01)
ax.set_xlim(2020, 2100)
ax.set_ylabel('°C relative to HFC-410A baseline')


mkdir_p('../plots/')
fig.set_facecolor('white')
pl.tight_layout()
pl.savefig('../plots/scenario-comparisons-temperature.png')